# Import Libraries

In [32]:
import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Import Datasets

*payment.csv*     : **paymentdf**  
*loan.csv*        : **loandf**   
*clarity_underwriting_variables.csv* : **claritydf**  

In [33]:
subdirectory = "data"
payment_path = os.path.join(subdirectory, "payment.csv")
loan_path = os.path.join(subdirectory, "loan.csv")
clarity_path = os.path.join(subdirectory, "clarity_underwriting_variables.csv")

In [34]:
paymentdf = pd.read_csv(payment_path)
loandf = pd.read_csv(loan_path)
claritydf = pd.read_csv(clarity_path)


/var/folders/4g/jfvz9fnd7blddks5n_g5qm3r0000gn/T/ipykernel_59902/3373870957.py:1: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,31,32,33,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  claritydf = pd.read_csv(clarity_path)


# Explore Data

In [49]:
# Show all the columns name of each dataframe
print("paymentdf columns:\n", paymentdf.columns)
print("\nloandf columns:\n", loandf.columns)
print("\nclaritydf columns:\n", claritydf.columns)

paymentdf columns:
 Index(['loanId', 'installmentIndex', 'isCollection', 'paymentDate',
       'principal', 'fees', 'paymentAmount', 'paymentStatus',
       'paymentReturnCode'],
      dtype='object')

loandf columns:
 Index(['loanId', 'anon_ssn', 'payFrequency', 'apr', 'applicationDate',
       'originated', 'originatedDate', 'nPaidOff', 'approved', 'isFunded',
       'loanStatus', 'loanAmount', 'originallyScheduledPaymentAmount', 'state',
       'leadType', 'leadCost', 'fpStatus', 'clarityFraudId', 'hasCF'],
      dtype='object')

claritydf columns:
 Index(['.underwritingdataclarity.clearfraud.clearfraudinquiry.thirtydaysago',
       '.underwritingdataclarity.clearfraud.clearfraudinquiry.twentyfourhoursago',
       '.underwritingdataclarity.clearfraud.clearfraudinquiry.oneminuteago',
       '.underwritingdataclarity.clearfraud.clearfraudinquiry.onehourago',
       '.underwritingdataclarity.clearfraud.clearfraudinquiry.ninetydaysago',
       '.underwritingdataclarity.clearfraud.clearf

In [52]:
# Show the data types of each column of loandf
print("\n------------Data Types-----------------")
print(loandf.dtypes)
print("\n--------------Shape--------------------")
print("Shape:", loandf.shape)

# Count missing values for each column
print("\n------------Missing Values-----------------")
print(loandf.isnull().sum())

# Count number of unique values for each column
print("\n------------Unique Values-----------------")
print(loandf.nunique())

# loandf data summary
print("\n------------Summary-----------------")
print(loandf.describe())

# Show the first 5 rows of loandf
print("\n------------First 5 rows-----------------")
loandf.head()


------------Data Types-----------------
loanId                               object
anon_ssn                             object
payFrequency                         object
apr                                 float64
applicationDate                      object
originated                             bool
originatedDate                       object
nPaidOff                            float64
approved                               bool
isFunded                              int64
loanStatus                           object
loanAmount                          float64
originallyScheduledPaymentAmount    float64
state                                object
leadType                             object
leadCost                              int64
fpStatus                             object
clarityFraudId                       object
hasCF                                 int64
dtype: object

--------------Shape--------------------
Shape: (577682, 19)

------------Missing Values-----------------
loa

,loanId,anon_ssn,payFrequency,apr,applicationDate,originated,originatedDate,nPaidOff,approved,isFunded,loanStatus,loanAmount,originallyScheduledPaymentAmount,state,leadType,leadCost,fpStatus,clarityFraudId,hasCF
0,LL-I-07399092,beff4989be82aab4a5b47679216942fd,B,360.0,2016-02-23T17:29:01.940000,False,NaN,0.0,False,0,Withdrawn Application,500.0,978.27,IL,bvMandatory,6,NaN,5669ef78e4b0c9d3936440e6,1
1,LL-I-06644937,464f5d9ae4fa09ece4048d949191865c,B,199.0,2016-01-19T22:07:36.778000,True,2016-01-20T15:49:18.846000,0.0,True,1,Paid Off Loan,3000.0,6395.19,CA,prescreen,0,Checked,569eb3a3e4b096699f685d64,1
2,LL-I-10707532,3c174ae9e2505a5f9ddbff9843281845,B,590.0,2016-08-01T13:51:14.709000,False,NaN,0.0,False,0,Withdrawn Application,400.0,1199.45,MO,bvMandatory,3,NaN,579eab11e4b0d0502870ef2f,1
3,LL-I-02272596,9be6f443bb97db7e95fa0c281d34da91,B,360.0,2015-08-06T23:58:08.880000,False,NaN,0.0,False,0,Withdrawn Application,500.0,1074.05,IL,bvMandatory,3,NaN,555b1e95e4b0f6f11b267c18,1
4,LL-I-09542882,63b5494f60b5c19c827c7b068443752c,B,590.0,2016-06-05T22:31:34.304000,False,NaN,0.0,False,0,Rejected,350.0,814.37,NV,bvMandatory,3,NaN,5754a91be4b0c6a2bf424772,1


# Data Preprocessing

## Preprocess loandf / loan.csv   
   
Potentially unuseful columns
- *originatedDate*
    - Only include it, if we are considering the timeline application is submitted
- *isFunded*
    - The loan is ultimately funded depends on the willingness of candidates, thus, it does not affect the qualification of a candidate.

Notes:
- Consider relationship of *loanStatus* and *approved* 
    - Even though the *approved* is false, but it might due to application withdrawal, which doesn't mean he/she a unqualified candidate

Target Variable
- *loanStatus* will be the first prioritised target value

Question:
- What is the relationship of *loanAmount* and *originallyScheduledPaymentAmount*?


### Preprocess Target Variable - *loanStatus*

In [54]:
# Count every unique values in column loanStatus
print(loandf["loanStatus"].value_counts())

loanStatus
Withdrawn Application          450984
Rejected                        85070
Paid Off Loan                   11427
External Collection             11334
New Loan                         8112
Internal Collection              5567
Returned Item                    1182
CSR Voided New Loan              1026
Settlement Paid Off               708
Credit Return Void                704
Customer Voided New Loan          504
Settled Bankruptcy                326
Pending Paid Off                  169
Charged Off Paid Off              160
Pending Application Fee             5
Pending Rescind                     4
Pending Application                 4
Voided New Loan                     2
Charged Off                         1
Customver Voided New Loan           1
Settlement Pending Paid Off         1
Name: count, dtype: int64


We can see that there are many classes to be predicted. We will study and remove the unwanted classes.

1. We will remove "Withdrawn Application" because these applications are not yet classified thus it is meaningless.

2. We will remove small classes that frequency has lower than 100.

3. We will combine "External Collection" and "Internal Collection" as a single value, "Debt Collection".

### Remove unwanted columns  
Columns to be dropped: 
- *originatedDate*
- *isFunded*


In [ ]:
# Drop originatedDate and isFunded columns
loandf.drop(["originatedDate", "isFunded"], axis=1, inplace=True)

### Impute missing value
- nPaidOff: Impute value '0' if missing
- 

In [44]:
# Count missing values for each column
print("\n------------Missing Values-----------------")
print(loandf.isnull().sum())


------------Missing Values-----------------
loanId                                 256
anon_ssn                                 0
payFrequency                          1273
apr                                   3922
applicationDate                          0
originated                               0
originatedDate                      531638
nPaidOff                                24
approved                                 0
isFunded                                 0
loanStatus                             391
loanAmount                            2250
originallyScheduledPaymentAmount         0
state                                  132
leadType                                 0
leadCost                                 0
fpStatus                            525959
clarityFraudId                      219989
hasCF                                    0
dtype: int64


In [ ]:
# Address missing values in column nPaidOff
# We impute integer -1 because all records of NA in nPaidOff had been approved for loan
loandf["nPaidOff"].replace("NA", -1, inplace=True) # Replace string value "NA" with 0
loandf["nPaidOff"].fillna(-1, inplace=True) # Replace null value with 0

### Remove unwanted rows

In [ ]:
# Remove rows with loanAmount is N/A

## Preprocess paymentdf / payment.csv

## Preprocess claritydf / clarity_underwriting_variables.csv

# Data Modeling

# Model Evaluation

We will use "False Negative" as our metrics to evaluate our models, because we want to reduce financial impact of lending money to borrower who will default the loan.